In [ ]:
inputdir = '/facility/imganfacusers/Ashesh/NatureMethodsSegmentation/2405_D18-M3-S0-L8_13/'
segmentationdir = '/facility/imganfacusers/Ashesh/NatureMethodsSegmentationOutputs/Analysis_2405_D18-M3-S0-L8_13_1/'

OUTPUT_DIR = '/group/jug/ashesh/naturemethods/segmentation/three_analysts/'


In [ ]:
from disentangle.core.tiff_reader import load_tiff
import os
ch_idx = 1 # which channel are we interested in doing segmentation.
gt = load_tiff(os.path.join(inputdir, 'GT.tif'))
pred  = load_tiff(os.path.join(inputdir, 'pred_disentangle_2405_D18-M3-S0-L8_13_1.tif'))
pred.shape

In [ ]:
input = gt[:,0] + gt[:,1]
gt_ch = gt[:,ch_idx]
pred_ch = pred[:,ch_idx]
raw_dict = {'input':input, 'pred':pred_ch, 'GT':gt_ch}


In [ ]:
from collections import defaultdict
def load_segmentation():
    # 2 level nested dictionary
    segmentation = defaultdict(lambda: defaultdict(list))
    for analystdir in os.listdir(segmentationdir):
        for subdir in ['GT','input','pred']:
            dirpath = os.path.join(segmentationdir, analystdir, subdir)
            fnames  = [f'seg_{i}.tif' for i in range(0,5)]
            fpaths = [os.path.join(dirpath, fname) for fname in fnames]
            for fpath in fpaths:
                segmentation[analystdir][subdir].append(load_tiff(fpath))
    return segmentation

In [ ]:
seg_data = load_segmentation()
analysts  = list(seg_data.keys())

In [ ]:
# import matplotlib.pyplot as plt
# from disentangle.analysis.plot_utils import clean_ax
# ex_idx =4
# ncols=4
# nrows = 3
# subres = 4

# imgsz = 3
# _, ax = plt.subplots(figsize=(ncols*imgsz, nrows*imgsz), ncols=ncols, nrows=nrows)

# for d_idx, datatype in enumerate(['input','pred','GT']):
#     ax[d_idx,0].imshow(raw_dict[datatype][ex_idx][::subres,::subres], cmap='magma')
#     for a_idx, analyst in enumerate(analysts):
#         ax[d_idx, 1+a_idx].imshow(seg_data[analyst][datatype][ex_idx][::subres,::subres], cmap='gray')
#         # ax[d_idx, 1+a_idx].set_title(f'{analyst} {datatype}')

# clean_ax(ax)
# # remove subspaces between subplots
# plt.subplots_adjust(wspace=0.02, hspace=0.02)


In [ ]:
import matplotlib.pyplot as plt
from disentangle.analysis.plot_utils import clean_ax

ex_idx =4
transpose = False
save_to_file = True



ncols=4
nrows = 3
subres = 4
# how much do we want to reduce the width of the image
width_factor = 5/12
new_w = int(gt.shape[-2]*width_factor)
new_w += (gt.shape[-2] - new_w)%2
extra_w = (gt.shape[-2] - new_w)//2
imgsz = 3
_, ax = plt.subplots(figsize=(ncols*imgsz*width_factor, nrows*imgsz), ncols=ncols, nrows=nrows)

for d_idx, datatype in enumerate(['input','pred','GT']):
    raw_img = raw_dict[datatype][ex_idx]
    if transpose:
        raw_img = raw_img.T
    ax[d_idx,0].imshow(raw_img[:,extra_w:-extra_w][::subres,::subres], cmap='magma')
    for a_idx, analyst in enumerate(analysts):
        seg_img = seg_data[analyst][datatype][ex_idx]
        if transpose:
            seg_img = seg_img.T
        ax[d_idx, 1+a_idx].imshow(seg_img[:,extra_w:-extra_w][::subres,::subres], cmap='gray', vmin=0, vmax=1)
        # ax[d_idx, 1+a_idx].set_title(f'{analyst} {datatype}')

clean_ax(ax)
# remove subspaces between subplots
plt.subplots_adjust(wspace=0.02, hspace=0.02)
if save_to_file:
    model_token = os.path.basename(inputdir.strip('/'))
    fname = f'segmentation_3analyst_Idx{ex_idx}_{model_token}.png'
    fpath = os.path.join(OUTPUT_DIR, fname)
    print(fpath)
    plt.savefig(fpath, dpi = 100, bbox_inches='tight')
